In [63]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import copy
# downlaod some necessary data
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Error loading punkt: <urlopen error [WinError 10061] No
[nltk_data]     connection could be made because the target machine
[nltk_data]     actively refused it>
[nltk_data] Error loading stopwords: <urlopen error [WinError 10061]
[nltk_data]     No connection could be made because the target machine
[nltk_data]     actively refused it>


False

In [57]:
train_data = pd.read_csv('train.txt',delimiter=' ::: ', engine='python')
test_data = pd.read_csv('test.txt', delimiter= ' ::: ', engine= 'python')

Importing train data and
Preprocessing

In [64]:
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)
preprocess_data = copy.deepcopy(train_data)
preprocess_data['description'] = preprocess_data['description'].apply(preprocess_text)

In [66]:
print(train_data['description'][1])
print(preprocess_data['description'][1])

eighteen yearold alex 13 yearold maggie four year old kay happy mothers new union nameless violent bluecollar drunk agree try make work suddenly killed car crash alex forced mature quickly look sisters promised looking move better life drops college works fulltime grocery store making minimum wage meanwhile barrier stepfathers bitter resentment grows antagonistic maggie verge womanhood kay beginning learn ways world alex grows increasingly protective sisters innocence support system girlfriend courtney threat prison abandoning sisters keeping alex violence far willing go keep family together
eighteen yearold alex 13 yearold maggie four year old kay happy mothers new union nameless violent bluecollar drunk agree try make work suddenly killed car crash alex forced mature quickly look sisters promised looking move better life drops college works fulltime grocery store making minimum wage meanwhile barrier stepfathers bitter resentment grows antagonistic maggie verge womanhood kay beginnin

In [41]:
data = preprocess_data

In [67]:
def compute_tf(text):
    tf_text = Counter(text.split())
    for i in tf_text:
        tf_text[i] = tf_text[i] / float(len(text.split()))
    return tf_text

def compute_idf(word, corpus):
    n_documents_with_word = sum([1 for document in corpus if word in document])
    if n_documents_with_word == 0:
        return 0
    return np.log10(len(corpus) / n_documents_with_word)

corpus = data['description'].tolist()
tfidf_matrix = []

for text in corpus:
    tfidf_text = []
    computed_tf = compute_tf(text)
    for word in text.split():
        tfidf = computed_tf[word] * compute_idf(word, corpus)
        tfidf_text.append(tfidf)
    tfidf_matrix.append(tfidf_text)

max_len = max(len(row) for row in tfidf_matrix)
tfidf_matrix = [row + [0] * (max_len - len(row)) for row in tfidf_matrix]
tfidf_matrix = np.array(tfidf_matrix)


In [68]:
def euclidean_distance(a, b):
    return np.sqrt(np.sum((a - b) ** 2))

def knn_predict(X_train, y_train, x_test, k=5):
    distances = []
    for i in range(len(X_train)):
        dist = euclidean_distance(X_train[i], x_test)
        distances.append((dist, y_train.iloc[i]))
    distances.sort(key=lambda x: x[0])
    neighbors = distances[:k]
    output_values = [neighbor[1] for neighbor in neighbors]
    prediction = Counter(output_values).most_common(1)[0][0]
    return prediction

# تقسیم داده‌ها به آموزشی و ارزیابی
def train_test_split(X, y, test_size=0.2, random_state=42):
    np.random.seed(random_state)
    indices = np.random.permutation(len(X))
    test_size = int(len(X) * test_size)
    test_indices = indices[:test_size]
    train_indices = indices[test_size:]
    X_train = X[train_indices]
    X_val = X[test_indices]
    y_train = y.iloc[train_indices]
    y_val = y.iloc[test_indices]
    return X_train, X_val, y_train, y_val

X_train, X_val, y_train, y_val = train_test_split(tfidf_matrix, data['genre'], test_size=0.2, random_state=42)


In [69]:
def classification_report(y_true, y_pred):
    labels = list(set(y_true))
    report = {}
    for label in labels:
        true_positive = sum((y_true == label) & (y_pred == label))
        false_positive = sum((y_true != label) & (y_pred == label))
        false_negative = sum((y_true == label) & (y_pred != label))
        precision = true_positive / (true_positive + false_positive) if true_positive + false_positive > 0 else 0
        recall = true_positive / (true_positive + false_negative) if true_positive + false_negative > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
        report[label] = {
            'precision': precision,
            'recall': recall,
            'f1-score': f1_score
        }
    return report

y_pred = []
for i in range(len(X_val)):
    y_pred.append(knn_predict(X_train, y_train, X_val[i]))

print(classification_report(np.array(y_val), np.array(y_pred)))


{'comedy': {'precision': 0.14285714285714285, 'recall': 0.10344827586206896, 'f1-score': 0.12000000000000001}, 'family': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0}, 'romance': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0}, 'animation': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0}, 'adventure': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0}, 'sci-fi': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0}, 'biography': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0}, 'crime': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0}, 'documentary': {'precision': 0.2532051282051282, 'recall': 0.32916666666666666, 'f1-score': 0.286231884057971}, 'reality-tv': {'precision': 0.5, 'recall': 0.0625, 'f1-score': 0.1111111111111111}, 'horror': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0}, 'drama': {'precision': 0.25635103926096997, 'recall': 0.4475806451612903, 'f1-score': 0.32599118942731276}, 'fantasy': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0}, 'music': {'precision': 0.0, 'recall

In [80]:
def preprocess_and_vectorize(description, corpus):
    description = preprocess_text(description)
    tfidf_text = []
    computed_tf = compute_tf(description)
    for word in description.split():
        tfidf = computed_tf[word] * compute_idf(word, corpus)
        tfidf_text.append(tfidf)
    if len(tfidf_text) < max_len:
        tfidf_text.extend([0] * (max_len - len(tfidf_text)))
    return np.array(tfidf_text)

def predict_genre(description, X_train, y_train, corpus):
    description_vector = preprocess_and_vectorize(description, corpus)
    predicted_genre = knn_predict(X_train, y_train, description_vector, k=5)
    return predicted_genre

# نمونه‌ای از توضیحات جدید برای آزمایش
new_description = train_data['description'][2]
predicted_genre = predict_genre(new_description, X_train, y_train, corpus)
print(f"Predicted Genre: {predicted_genre}")
print(train_data['genre'][2])


Predicted Genre: documentary
horror
